<a href="https://colab.research.google.com/github/Neko-Kuroi/mise_elixir_erlang_cloudflared_tunnel/blob/main/mise_elixir_erlang_cloudflared_tunnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! sudo apt update -y && sudo apt install -y gpg sudo wget curl
! sudo install -dm 755 /etc/apt/keyrings
! wget -qO - https://mise.jdx.dev/gpg-key.pub | gpg --dearmor | sudo tee /etc/apt/keyrings/mise-archive-keyring.gpg 1> /dev/null
! echo "deb [signed-by=/etc/apt/keyrings/mise-archive-keyring.gpg arch=amd64] https://mise.jdx.dev/deb stable main" | sudo tee /etc/apt/sources.list.d/mise.list
! sudo apt update
! sudo apt install -y mise

In [ ]:
# Erlangのインストール
! mise use -g erlang@27.2

# Elixirのインストール
! mise use -g elixir@1.18.1-otp-27

In [ ]:
import os
os.environ['PATH'] = "/root/.local/share/mise/shims:/root/.mix/escripts:" + os.environ['PATH']

In [ ]:
! mix local.hex --force
! mix local.rebar --force
! mix escript.install hex livebook --force
! livebook -v

In [ ]:
! elixir -v

In [ ]:
#%%writefile setup_livebookapp.py
import time
import os
#import hashlib
#import tempfile
import time
import base64
import subprocess
import threading
import sys
import re
from IPython.display import HTML, display
import logging
import requests # Import the requests library

# pass
os.environ['LIVEBOOK_PASSWORD'] = 'kurowoprogrammer'

# ログの設定
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

#ログ読み取り関数は削除またはコメントアウト
def read_process_output(process, stream, prefix=""):
    """プロセスの出力をリアルタイムで読み取り、表示する関数"""
    while True:
        line = stream.readline()
        if line:
            print(f"{prefix}{line.strip()}")
        elif process.poll() is not None:
            try:
                remaining = stream.read()
                if remaining:
                     print(f"{prefix}{remaining.strip()}")
            except ValueError:
                 pass
            break
        else:
            time.sleep(0.01)
    try:
        stream.close()
    except Exception as e:
        logger.debug(f"Error closing stream in read_process_output: {e}")


def setup_bore_tunnel():
    """Rust製のboreトンネルの設定"""
    print("🦀 Bore トンネルをセットアップしています...")

    # boreのダウンロードとインストール
    # -nc オプションは、ファイルが既に存在する場合に再ダウンロードしないようにします。
    os.system('wget -nc https://github.com/ekzhang/bore/releases/download/v0.6.0/bore-v0.6.0-x86_64-unknown-linux-musl.tar.gz')
    # ダウンロードしたアーカイブを解凍します。
    os.system('tar -zxvf bore-v0.6.0-x86_64-unknown-linux-musl.tar.gz')
    # 実行権限を付与。
    os.system('chmod 764 bore')

    # livebookアプリケーションをバックグラウンドで起動します。
    # stdoutとstderrをsubprocess.PIPEにリダイレクトしない
    print("🚀 livebook アプリケーションを起動しています...")
    livebook_process = subprocess.Popen(
        #["gunicorn", "--bind", "0.0.0.0:5000", "app:app", "--access-logfile", "-", "--error-logfile", "-"],
        ['livebook', 'server', '--port', '8888'],
        stdout=subprocess.PIPE, # ログ出力しないため削除
        stderr=subprocess.PIPE, # ログ出力しないため削除
        text=True,
        bufsize=1
    )

    # livebookプロセスの出力を読み取るスレッドは削除
    livebook_stdout_thread = threading.Thread(target=read_process_output, args=(livebook_process, livebook_process.stdout))
    livebook_stderr_thread = threading.Thread(target=read_process_output, args=(livebook_process, livebook_process.stderr))
    livebook_stdout_thread.daemon = True
    livebook_stderr_thread.daemon = True
    livebook_stdout_thread.start()
    livebook_stderr_thread.start()


    time.sleep(10)  # livebookサーバーが完全に起動するまで少し長めに7秒待ちます。

    # boreトンネルの起動
    print("🌐 bore トンネルを開始しています...")
    # boreをローカルポート5000からbore.pubへのトンネルとして起動します。
    # stdoutとstderrをsubprocess.PIPEにリダイレクトし、テキストモードでキャプチャします。
    bore_process = subprocess.Popen(['./bore', 'local', '8888', '--to', 'bore.pub'],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE,
                                   text=True,
                                   bufsize=1 # 行バッファリングを無効にしてリアルタイム出力を試みる
                                   )

    # Boreプロセスの出力を読み取るスレッドは削除
    # bore_stdout_thread = threading.Thread(target=read_process_output, args=(bore_process, bore_process.stdout, "[BORE_OUT] "))
    # bore_stderr_thread = threading.Thread(target=read_process_output, args=(bore_process, bore_process.stderr, "[BORE_ERR] "))
    # bore_stdout_thread.daemon = True
    # bore_stderr_thread.daemon = True
    # bore_stdout_thread.start()
    # bore_stderr_thread.start()


    # トンネルのURLを取得
    print("🔍 トンネルURLを待機しています...")
    url_found = False
    url = ""
    # タイムアウトを設定し、無限ループにならないようにします
    start_time = time.time()
    timeout = 130 # タイムアウト

    # Boreの標準出力からURLを読み取るループは維持（URL表示のため）
    while time.time() - start_time < timeout:
        line = bore_process.stdout.readline()
        if line:
            # print(f"Bore stdout: {line.strip()}") # デバッグ出力は削除
            match = re.search(r'(bore\.pub:\d+)', line)
            if match:
                extracted_url_part = match.group(0).strip()
                url = f"{extracted_url_part}"
                url_found = True
                break

        # boreが終了したか確認（エラーになった場合など）
        if bore_process.poll() is not None:
            print("Boreプロセスが予期せず終了しました。")
            break

        time.sleep(0.1) # 短い間隔で繰り返し確認

    if url_found:
        print(f"✅ トンネルが開始されました: {url}")
        # IPython.display.HTML を使用して、クリック可能なURLを生成し表示します。
        display(HTML(f'<a href="http://{url}" target="_blank" style="font-size:18px; color:pink;">{url}</a>'))
        # boreトンネルへのcurlテスト
        print("\n--- Boreトンネルへの内部curlテスト ---")
        try:
            # 外部からアクセスするURLに対してcurlを実行
            curl_url = f"http://{url}"
            curl_result = subprocess.run(['curl', '-s', '-I', curl_url], capture_output=True, text=True, timeout=10)
            print("curl出力（ヘッダのみ）：")
            print(curl_result.stdout.strip())
            if "200 OK" in curl_result.stdout:
                print("✅ curlテスト成功: HTTP 200 OK を受信しました。ブラウザでアクセスできるはずです。")
            else:
                print("❌ curlテスト失敗: 予期しない応答コードを受信しました。")
                # print(f"Boreの応答全文:\n{curl_result.stdout}\n{curl_result.stderr}") # スレッドが出力済みのため不要
        except subprocess.TimeoutExpired:
            print("❌ curlテスト失敗: タイムアウトしました。Boreサービスが応答していません。")
        except Exception as e:
            print(f"❌ curlテスト中にエラーが発生しました: {e}")
        print("----------------------------")

    else:
        print("⚠️ トンネルURLの取得に失敗しました。")
        # Boreプロセスの標準出力と標準エラー出力の表示は削除
        # print("Boreプロセスの標準出力と標準エラー出力を確認してください。")
        # final_stdout = bore_process.stdout.read()
        # final_stderr = bore_process.stderr.read()
        # if final_stdout:
        #     print(f"最終的なBore stdout: \n{final_stdout.strip()}")
        # if final_stderr:
        #     print(f"最終的なBore stderr: \n{final_stderr.strip()}")

        # スレッドのjoinも不要
        # bore_stdout_thread.join(timeout=5)
        # bore_stderr_thread.join(timeout=5)


    return livebook_process, bore_process

def setup_cloudflare_tunnel():
    """Cloudflare Tunnelの設定"""
    print("☁️ Cloudflare Tunnel をセットアップしています...")

    # cloudflaredのダウンロードとインストール
    os.system('wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb')
    os.system('dpkg -i cloudflared-linux-amd64.deb 2>/dev/null')

    # livebookアプリケーションをバックグラウンドで起動
    # stdoutとstderrをsubprocess.PIPEにリダイレクトしない
    print("🚀 livebook アプリケーションを起動しています...")
    livebook_process = subprocess.Popen(
        #["gunicorn", "--bind", "0.0.0.0:5000", "app:app", "--access-logfile", "-", "--error-logfile", "-"],
        ['livebook', 'server', '--port', '8888'],
        stdout=subprocess.PIPE, # ログ出力しないため削除
        stderr=subprocess.PIPE, # ログ出力しないため削除
        text=True,
        bufsize=1
    )

    # livebookプロセスの出力を読み取るスレッドは削除
    livebook_stdout_thread = threading.Thread(target=read_process_output, args=(livebook_process, livebook_process.stdout))
    livebook_stderr_thread = threading.Thread(target=read_process_output, args=(livebook_process, livebook_process.stderr))
    livebook_stdout_thread.daemon = True
    livebook_stderr_thread.daemon = True
    livebook_stdout_thread.start()
    livebook_stderr_thread.start()

    time.sleep(10)  # サーバーの起動を待つ

    # Cloudflareトンネルの起動
    print("🌐 Cloudflare トンネルを開始しています...")
    tunnel_process = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:8888'],
                                     stdout=subprocess.PIPE,
                                     stderr=subprocess.PIPE,
                                     text=True,
                                     bufsize=1)

    # Cloudflare Tunnelプロセスの出力を読み取るスレッドは削除
    # tunnel_stdout_thread = threading.Thread(target=read_process_output, args=(tunnel_process, tunnel_process.stdout, "[CF_OUT] "))
    # tunnel_stderr_thread = threading.Thread(target=read_process_output, args=(tunnel_process, tunnel_process.stderr, "[CF_ERR] "))
    # tunnel_stdout_thread.daemon = True
    # tunnel_stderr_thread.daemon = True
    # tunnel_stdout_thread.start()
    # tunnel_stderr_thread.start()


    # cloudflaredの出力からURLを抽出するループは維持
    url_found = False
    url = ""
    start_time = time.time()
    timeout = 130

    while time.time() - start_time < timeout:
        line = tunnel_process.stderr.readline() # Cloudflare TunnelはstderrにURLを出す傾向がある
        if line:
            # print(f"Cloudflare Tunnel stderr: {line.strip()}") # デバッグ出力は削除
            if 'https://' in line and 'trycloudflare.com' in line:
                match = re.search(r'(https:\/\/[^\s]+\.trycloudflare\.com)', line)
                if match:
                    url = match.group(0).strip()
                    url_found = True
                    break

        if tunnel_process.poll() is not None:
            print("Cloudflare Tunnelプロセスが予期せず終了しました。")
            break

        time.sleep(0.1)

    if url_found:
        print(f"✅ Cloudflare トンネルが開始されました: {url}")
        display(HTML(f'<a href="{url}" target="_blank" style="font-size:18px; color:pink;">ファイルアップロードサービスにアクセス: {url}</a>'))
    else:
        print("⚠️ CloudflareトンネルURLの取得に失敗しました。")
        # Cloudflare Tunnelプロセスの標準出力と標準エラー出力の表示は削除
        # print("Cloudflare Tunnelプロセスの標準出力と標準エラー出力を確認してください。")
        # final_stdout = tunnel_process.stdout.read()
        # final_stderr = tunnel_process.stderr.read()
        # if final_stdout:
        #     print(f"最終的なCloudflare Tunnel stdout: \n{final_stdout.strip()}")
        # if final_stderr:
        #     print(f"最終的なCloudflare Tunnel stderr: \n{final_stderr.strip()}")

        # スレッドのjoinも不要
        # tunnel_stdout_thread.join(timeout=5)
        # tunnel_stderr_thread.join(timeout=5)


    return livebook_process, tunnel_process

def wait_for_livebook_server(port=8888, timeout=15):
    """livebookサーバーが起動し、リクエストに応答するのを待機します。"""
    url = f"http://localhost:{port}"
    print(f"Waiting for server to start at {url}...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            response = requests.get(url, timeout=1)
            if response.status_code == 200:
                print(f"✅ livebook server is up and running! Status code: {response.status_code}")
                return True
        except requests.exceptions.ConnectionError:
            pass # サーバーがまだ起動していない、または接続を拒否している
        except Exception as e:
            print(f"Error during livebook server health check: {e}")
            pass
        time.sleep(1)
    print(f"❌ livebook server did not respond within {timeout} seconds.")
    return False


def get_colab_external_ip():
    """Colabの外部IPアドレスを取得します。"""
    try:
        result = subprocess.run(['curl', 'ipinfo.io/ip'], capture_output=True, text=True, check=True)
        return result.stdout.strip()
    except Exception as e:
        print(f"⚠️ 外部IPアドレスの取得に失敗しました: {e}")
        return "UNKNOWN_IP"

selected_tunnel_service = ""
while True:
    print("\n--- トンネル方法を選択してください ---")
    print("1. Bore")
    print("2. Cloudflared")
    #print("3. Localtunnel (Node.js/npm が必要です)")
    choice = input("選択 (1/2): ").strip()

    if choice == '1':
        selected_tunnel_service = "bore"
        break
    elif choice == '2':
        selected_tunnel_service = "cloudflared"
        break
    else:
        print("無効な選択です。1 または 2 を入力してください。")


if selected_tunnel_service == "bore":
    livebook_process, bore_process = setup_bore_tunnel()
    # bore_process.wait() # waitは削除
elif selected_tunnel_service == "cloudflared":
    livebook_process, cloudflared_process = setup_cloudflare_tunnel()
    tunnel_process.wait() # waitは削除

# プロセスがバックグラウンドで実行され続けるように、メインスレッドは特に待機しない
# Colabのセルが実行中である限り、子プロセスは実行を続けます。
# セルの実行が完了すると、これらのプロセスも通常終了します。

print(f"\nColab 外部IPアドレス: {get_colab_external_ip()}")

## livebook

In [ ]:
# livebook
Mix.install(
  [
    {:finch, "~> 0.16"},
    {:floki, "~> 0.34"},
    {:calendar, "~> 1.0"},
    {:kino, "~> 0.16"},
    {:pythonx, "~> 0.4.2"},
    {:kino_pythonx, "~> 0.1.0"}
  ],
  consolidate_protocols: false
)

In [ ]:
# livebook python enable
# pyproject.toml
[project]
name = "project"
version = "0.0.0"
requires-python = "==3.13.*"
dependencies = []

In [ ]:
# livebook
# # 依存関係のインストールと初期設定
# Mix.install([
#   {:finch, "~> 0.16"},
#   {:floki, "~> 0.34"},
#   {:calendar, "~> 1.0"},
#   {:kino, "~> 0.10"}
# ], consolidate_protocols: false)

# モジュールの定義
defmodule HatenaBookmarks do
  @moduledoc """
  Livebook対応版Hatenaブックマークスクレイパー
  """

  # 設定構造体
  defmodule Config do
    defstruct [
      :base_url,
      :categories,
      :exclude_domains,
      :no_description_domains,
      :max_description_length,
      :request_delay_ms,
      :timeout_ms,
      :max_retries,
      :output_directory
    ]

    def default do
      %__MODULE__{
        base_url: "https://b.hatena.ne.jp/entrylist/",
        categories: [
          "social", "general", "life", "knowledge",
          "fun", "game", "economics", "entertainment"
        ],
        exclude_domains: ~w[
          automation-media abema soredoko onaji\.me
          hatena hatelabo togetter denfaminicogamer
          shonenjump dailyshincho posfie toyokeizai
        ],
        no_description_domains: ~w[
          asahi shonenjump dailyshincho
          posfie togetter toyokeizai
        ],
        max_description_length: 400,
        request_delay_ms: 1000,
        timeout_ms: 30_000,
        max_retries: 3,
        output_directory: System.tmp_dir!()
      }
    end
  end

  # ブックマーク構造体
  defmodule Bookmark do
    defstruct [
      :category,
      :title,
      :url,
      :date,
      :description,
      :fetched_at
    ]

    def new(attrs) do
      struct(__MODULE__, Map.put(attrs, :fetched_at, DateTime.utc_now()))
    end
  end

  # 結果集計構造体
  defmodule Results do
    defstruct [
      bookmarks: [],
      total_fetched: 0,
      total_unique: 0,
      errors: [],
      duration_ms: 0,
      output_file: nil
    ]
  end

  # 公開API
  def run(opts \\ []) do
    config = struct(Config.default(), opts)
    main(config)
  end

  # メイン処理 - with句の戻り値修正
  defp main(config) do
    start_time = System.monotonic_time(:millisecond)

    with :ok <- start_finch(),
         :ok <- ensure_output_directory(config.output_directory),
         results <- scrape_all_categories(config) do  # {:ok} タプルなしで直接取得

      end_time = System.monotonic_time(:millisecond)
      final_results = %{results | duration_ms: end_time - start_time}

      case save_results(final_results, config) do
        {:ok, file_path} ->
          log_final_results(final_results, file_path)
          {:ok, %{final_results | output_file: file_path}}

        {:error, reason} ->
          {:error, reason}
      end
    else
      {:error, reason} -> {:error, reason}
    end
  end

  # 内部関数
  defp start_finch do
    case Finch.start_link(name: HatenaBookmarksFinch) do
      {:ok, _pid} -> :ok
      {:error, {:already_started, _pid}} -> :ok
      {:error, reason} -> {:error, reason}
    end
  end

  defp ensure_output_directory(dir) do
    case File.mkdir_p(dir) do
      :ok -> :ok
      {:error, reason} -> {:error, reason}
    end
  end

  # 戻り値をResults構造体に変更
  defp scrape_all_categories(config) do
    results =
      config.categories
      |> Task.async_stream(
           &process_category_with_delay(&1, config),
           timeout: config.timeout_ms,
           on_timeout: :kill_task,
           max_concurrency: 2
         )
      |> Enum.reduce(%Results{}, fn
        {:ok, {:ok, bookmarks}}, acc ->
          %{acc |
            bookmarks: bookmarks ++ acc.bookmarks,
            total_fetched: acc.total_fetched + length(bookmarks)
          }

        {:ok, {:error, {category, reason}}}, acc ->
          error = "カテゴリ '#{category}' の処理に失敗: #{inspect(reason)}"
          %{acc | errors: [error | acc.errors]}

        {:exit, reason}, acc ->
          error = "タスクタイムアウト: #{inspect(reason)}"
          %{acc | errors: [error | acc.errors]}
      end)

    unique_bookmarks =
      results.bookmarks
      |> Enum.uniq_by(& &1.url)
      |> Enum.shuffle()

    %{results |
      bookmarks: unique_bookmarks,
      total_unique: length(unique_bookmarks)
    }
  end

  defp process_category_with_delay(category, config) do
    if category != hd(config.categories) do
      Process.sleep(config.request_delay_ms)
    end
    process_category(category, config)
  end

  defp process_category(category, config) do
    url = config.base_url <> category

    case fetch_with_retry(url, config) do
      {:ok, html} ->
        case parse_bookmarks(html, category, config) do
          {:ok, bookmarks} -> {:ok, bookmarks}
          {:error, reason} -> {:error, {category, reason}}
        end

      {:error, reason} ->
        {:error, {category, reason}}
    end
  end

  defp fetch_with_retry(url, config, attempt \\ 1) do
    headers = [
      {"User-Agent", "HatenaScraper/1.0 (Livebook)"},
      {"Accept", "text/html"},
      {"Accept-Language", "ja,en;q=0.9"}
    ]

    case Finch.build(:get, url, headers) |> Finch.request(HatenaBookmarksFinch) do
      {:ok, %Finch.Response{status: 200, body: body}} ->
        {:ok, body}

      {:ok, %Finch.Response{status: status}} when status in 400..499 ->
        {:error, {:http_client_error, status}}

      {:ok, %Finch.Response{status: status}} when status in 500..599 ->
        if attempt < config.max_retries do
          Process.sleep(attempt * 2000)
          fetch_with_retry(url, config, attempt + 1)
        else
          {:error, {:http_server_error, status}}
        end

      {:error, reason} ->
        if attempt < config.max_retries do
          Process.sleep(attempt * 2000)
          fetch_with_retry(url, config, attempt + 1)
        else
          {:error, {:network_error, reason}}
        end
    end
  end

  defp parse_bookmarks(html, category, config) do
    case Floki.parse_document(html) do
      {:ok, doc} ->
        bookmarks =
          doc
          |> Floki.find("div.entrylist-contents")
          |> Enum.reduce([], fn entry, acc ->
            case extract_bookmark_data(entry, category, config) do
              {:ok, bookmark} -> [bookmark | acc]
              {:error, _} -> acc
            end
          end)

        {:ok, bookmarks}

      {:error, reason} ->
        {:error, reason}
    end
  end

  defp extract_bookmark_data(entry, category, config) do
    with {:ok, link_data} <- extract_link_data(entry),
         {:ok, url} <- validate_url(link_data.href, config),
         {:ok, description} <- extract_description(entry, url, config),
         {:ok, date} <- extract_date(entry) do

      description_str = to_string(description)

      bookmark = Bookmark.new(%{
        category: category,
        title: format_title(link_data.title),
        url: url,
        date: date,
        description: description_str
      })

      {:ok, bookmark}
    else
      error -> error  # すべてのエラーを伝播
    end
  end

  defp extract_link_data(entry) do
    case Floki.find(entry, "a.js-keyboard-openable") do
      [{"a", attrs, _} | _] ->
        attrs_map = Enum.into(attrs, %{})
        href = Map.get(attrs_map, "href")
        title = Map.get(attrs_map, "title", "") |> String.trim()

        if href && title != "" do
          {:ok, %{href: href, title: title}}
        else
          {:error, :missing_link_data}
        end

      _ ->
        {:error, :no_link_found}
    end
  end

  defp validate_url(url, config) do
    cond do
      is_nil(url) ->
        {:error, :nil_url}

      should_exclude_url?(url, config) ->
        {:error, :excluded_url}

      true ->
        {:ok, "(#{url})"}
    end
  end

  defp should_exclude_url?(url, config) do
    String.ends_with?(url, ".pdf") or
      Enum.any?(config.exclude_domains, &Regex.match?(~r/#{&1}/i, url))
  end

  defp should_skip_description?(url, config) do
    Enum.any?(config.no_description_domains, &Regex.match?(~r/#{&1}/i, url))
  end

  defp extract_description(entry, url, config) do
    if should_skip_description?(url, config) do
      {:ok, ""}
    else
      description =
        case Floki.find(entry, "p.entrylist-contents-description") do
          [element | _] ->
            element
            |> Floki.text()
            |> String.trim()
            |> truncate(config.max_description_length)
          [] -> ""
        end

      {:ok, description}
    end
  end

  defp extract_date(entry) do
    date =
      case Floki.find(entry, "li.entrylist-contents-date") do
        [element | _] ->
          element
          |> Floki.text()
          |> String.trim()
        [] -> ""
      end

    {:ok, date}
  end

  defp format_title(title) do
    cleaned =
      title
      |> String.replace(~r/\s+/, " ")
      |> String.replace("　", " ")

    "[ #{cleaned} ]"
  end

  defp truncate(text, max_length) do
    if String.length(text) > max_length do
      String.slice(text, 0, max_length) <> " ..."
    else
      text
    end
  end

  defp format_bookmark(bookmark, index) do
    description_text =
      bookmark.description
      |> String.split("\n")
      |> Enum.map(&String.trim/1)
      |> Enum.reject(&(&1 == ""))
      #|> Enum.map(&(&1 <> "。"))
      |> Enum.join("")

    """

    ------------------------------------------------
    #{String.pad_leading(to_string(index), 4, "0")}
    **#{to_string(bookmark.title)}#{bookmark.url}**
    #{to_string(bookmark.date)}

    #{to_string(description_text)}
    """
  end

  defp save_results(results, config) do
    # タイムスタンプをフォーマット
    {{year, month, day}, {hour, minute, second}} = :calendar.universal_time()
    timestamp = "#{year}#{String.pad_leading(to_string(month), 2, "0")}#{String.pad_leading(to_string(day), 2, "0")}_#{String.pad_leading(to_string(hour), 2, "0")}#{String.pad_leading(to_string(minute), 2, "0")}#{String.pad_leading(to_string(second), 2, "0")}"

    filename = "hatena_bookmarks_#{timestamp}.txt"
    file_path = Path.join(config.output_directory, filename)


    header = """
    ================================================
    Hatena Bookmarks Export (Livebook)
    ================================================
    生成日: #{DateTime.utc_now()}
    取得件数: #{results.total_fetched}
    ユニーク件数: #{results.total_unique}
    処理時間: #{results.duration_ms}ms
    エラー数: #{length(results.errors)}
    ================================================

    """

    content =
      results.bookmarks
      |> Enum.with_index(1)
      |> Enum.map(fn {bookmark, index} ->
        format_bookmark(bookmark, index)
      end)
      |> Enum.join("")

    full_content = header <> content
    IO.puts(file_path)
    case File.write(file_path, full_content) do
      :ok -> {:ok, file_path}
      {:error, reason} -> {:error, reason}
    end
  end

  defp log_final_results(results, file_path) do
    IO.puts("""
    ✅ スクレイピング完了！
    📁 出力ファイル: #{file_path}
    📊 統計情報:
       - 取得件数: #{results.total_fetched}
       - ユニーク件数: #{results.total_unique}
       - 重複除外: #{results.total_fetched - results.total_unique}
       - 処理時間: #{results.duration_ms}ms
       - エラー件数: #{length(results.errors)}
    """)
  end

end

# 実行開始メッセージ
start_message = """
## Hatenaブックマークスクレイピングを開始します

- 対象カテゴリ数: #{length(HatenaBookmarks.Config.default().categories)}
- 取得中...（1-2分かかる場合があります）
"""

Kino.Markdown.new(start_message)

# 非同期でスクレイピングを実行
task = Task.async(fn ->
  HatenaBookmarks.run()
end)

# 結果の処理と表示
scraping_result = case Task.await(task, 300_000) do  # 5分タイムアウト
  {:ok, results} ->
    # 結果サマリー
    summary = """
    ## ✅ スクレイピング成功！

    **取得結果:**
    - カテゴリ数: #{length(HatenaBookmarks.Config.default().categories)}
    - 総取得件数: #{results.total_fetched}
    - ユニーク件数: #{results.total_unique}
    - 処理時間: #{results.duration_ms}ms
    - 出力ファイル: #{results.output_file}
    """

    Kino.Markdown.new(summary)

    # ダウンロードリンク
    Kino.Download.new(results.output_file,
      label: "結果をダウンロード"
    )

    # サンプル表示
    if length(results.bookmarks) > 0 do
      Kino.Markdown.new("### :")

      sample_content = results.bookmarks
        |> Enum.take(results.total_unique)
        |> Enum.with_index(1)  # インデックスを1から開始
        |> Enum.map(fn {bookmark, index} ->
          # 4桁のインデックス（0001形式）にフォーマット
          index_str = String.pad_leading(to_string(index), 4, "0")
        #|> Enum.map(fn bookmark ->
          """
          ----
          #{index_str}
          **#{bookmark.title}#{bookmark.url}**
          #{bookmark.date}

          #{String.slice(bookmark.description, 0..400)} ...
          """
        end)
        |> Enum.join("\n\n")

      # Kino.Markdown.new(sample_content)
      # IO.puts(sample_content)
    else
      Kino.Markdown.new("取得したブックマークは0件です。")
    end

  {:error, reason} ->
    error_message = """
    ## ❌ エラーが発生しました

    **原因:**
    #{inspect(reason)}

    設定を変更して再実行してみてください。
    """

    Kino.Markdown.new(error_message)
end


In [ ]:
# livebook python
import glob
import os
import shutil

path = os.chdir('/tmp/')#os.getcwd()
print(path)
#shutil.rmtree('/tmp/')
file_paths = glob.glob('*24.txt')
print(file_paths)
if len(file_paths) > 0:
  for file in file_paths:
    with open(file, 'r') as f:
      for line in f.readlines():

        print(line, end='')